In [1]:
#using Pkg
#Pkg.add(PackageSpec(name="JWAS",rev="master"))

In [2]:
#Pkg.develop(PackageSpec(path="/home/jovyan/rohan/Box Sync/JWAS.jl"))
#Pkg.free("JWAS")
#Pkg.add("StatsPlots")

In [1]:
using DataFrames              # package for working with data sets
using JWAS                    # package for Bayesian regression analyses, including BayesB and BayesCπ        
using JWAS:misc               # utility functions
using Distributions       
using Plots                   # package for plotting 
using LinearAlgebra,Statistics,Random,DelimitedFiles, DataFrames

┌ Info: Recompiling stale cache file /opt/julia/compiled/v1.1/JWAS/tbeXw.ji for JWAS [c9a035f4-d403-5e6b-8649-6be755bc4798]
└ @ Base loading.jl:1184


In [7]:
methods(fill)

# 6 methods for generic function "fill":
[1] fill(v, dims::Union{Integer, AbstractUnitRange}...) in Base at array.jl:401
[2] fill(v, dims::Tuple{}) in Base at array.jl:404
[3] fill(v, dims::Tuple{Vararg{Integer,N}}) where N in Base at array.jl:403
[4] fill(v, dims::Tuple{Vararg{Union{Integer, OneTo},N}}) where N in Base at array.jl:402
[5] fill(val, ::SA) where SA<:StaticArrays.StaticArray in StaticArrays at /opt/julia/packages/StaticArrays/VyRz3/src/arraymath.jl:29
[6] fill(val, ::Type{SA}) where SA<:StaticArrays.StaticArray in StaticArrays at /opt/julia/packages/StaticArrays/VyRz3/src/arraymath.jl:30

### Input marker and phenotype data

In [4]:
function readMatBin(fileName)
    genStr = open(fileName)
    n = read(genStr,Int64)
    p = read(genStr,Int64)
    M = zeros(n,p)
    for j in 1:p
        for i in 1:n
            M[i,j] = read(genStr,Float64)
        end
    end
    close(genStr)
    return M
end

function removeCols!(M,cols)
    return M[:, [!(i in cols) for i=1:size(M,2)]]
end

removeCols! (generic function with 1 method)

In [5]:
posQTL  = Int64.(vec(readdlm("posQTL.csv")))
beta    = readdlm("beta.csv")
M = readMatBin("genotypes.bin");

In [6]:
n,p = size(M)
simData  = readtable("phenotypes.csv",header=false,names=[:y])# reading in the simulated phenotypes into a data frame
phenData = DataFrame(id=1:n, y=simData[:y])
first(phenData,5)

┌ Warning: readtable is deprecated, use CSV.read from the CSV package instead
│   caller = ip:0x0
└ @ Core :-1


,id,y
,Int64,Float64⍰
1,1,-3.56863
2,2,1.73437
3,3,2.31795
4,4,-0.264018
5,5,-3.13096


In [7]:
phenTrain = phenData[1001:end,:]
first(phenTrain,5)

,id,y
,Int64,Float64⍰
1,1001,-6.22029
2,1002,-0.952557
3,1003,-9.66847
4,1004,-0.959437
5,1005,1.48486


In [8]:
resVar = var(simData[:y])/2
genVar = resVar

12.83523324152048

### Run BayesC$\pi$ using JWAS

In [9]:
ids = string.(1:size(M,1))                     # ids in genotype file are sequential numbers 1...n
model  = build_model("y = intercept",resVar)   # give model (except for marker part)
add_genotypes(model,M,genVar,header=false,rowID=ids,G_is_marker_variance=false);

21834 markers on 2000 individuals were added.


In [10]:
?runMCMC

search: runMCMC ZeroMeanFullNormalCanon



```
runMCMC(model::MME,df::DataFrame;
        chain_length=1000,starting_value=false,burnin = 0,
        output_samples_frequency = 0,output_samples_file="MCMC_samples",
        printout_model_info=true,printout_frequency=100,
        methods="conventional (no markers)",Pi=0.0,estimatePi=false,
        single_step_analysis= false,pedigree = false,
        missing_phenotypes=false,constraint=false,
        update_priors_frequency::Int64=0,
        outputEBV=true,output_PEV=false,output_heritability=false)
```

**Run MCMC for Bayesian Linear Mixed Models with or without estimation of variance components.**

  * Available **methods** include "conventional (no markers)", "RR-BLUP", "BayesB", "BayesC", "Bayesian Lasso", and "GBLUP".
  * Single step analysis is allowed if **single*step*analysis** = `true` and **pedigree** is provided.
  * The **starting_value** can be provided as a vector of numbers for all location parameteres and marker effects, defaulting to `0.0`s.
  * The first **burnin** iterations are discarded at the beginning of a MCMC chain of length **chain_length**.
  * Save MCMC samples every **output*samples*frequency** iterations, defaulting to `false`, to files **output*samples*file**, defaulting to `MCMC_samples.txt`. MCMC samples for hyperparametes (variance componets) and marker effects are saved by default if **output*samples*frequency** is provided. MCMC samples for location parametes can be saved using `output_MCMC_samples()`. Note that saving MCMC samples too frequently slows down the computation.
  * In Bayesian variable selection methods, **Pi** for single-trait analyses is a number; **Pi** for multi-trait analyses is a dictionary such as `Pi=Dict([1.0; 1.0]=>0.7,[1.0; 0.0]=>0.1,[0.0; 1.0]=>0.1,[0.0; 0.0]=>0.1)`, defaulting to `all markers have effects (0.0)` in single-trait analysis and `all markers have effects on all traits` in multi-trait analysis. **Pi** is estimated if **estimatePi** = true
  * In multi-trait analysis, **missing_phenotypes**, defaulting to `true`, and **constraint** variance components, defaulting to `false`, are allowed. If **constraint**=true, constrain residual covariances between traits to be zeros.
  * Print out the model information in REPL if `printout_model_info=true`; print out the monte carlo mean in REPL with **printout_frequency**, defaulting to `false`.
  * Individual estimted breeding values (EBVs) are returned if **outputEBV**=`true`, defaulting to `true`. Heritability and genetic variances are returned if **output_heritability**=`true`, defaulting to `false`. Note that estimation of heritability is computaionally intensive.


In [11]:
MCMCFileNAME = "MCMCSamples"                  # place to put samples of marker effects
                                              # marker effect is set to zero if that locus is not in model
out=runMCMC(model, phenTrain,                 # tell JWAS to run analysis, for given model and data 
    Pi=0.99,                                  # intial value of π
    estimatePi=true,
    chain_length=60000,                       # length of chain
    printout_frequency=5000,                  # how often to show progress of analysis 
    printout_model_info=true,                 # tell JWAS to show the options used in this analysis
    methods="BayesC",                         # tell JWAS to run a BayesC analysis
    output_samples_frequency=20,              # how often to output sampled quantities
    output_samples_file=MCMCFileNAME,         # file name to output sampled marker effects
    output_PEV=true
);


The prior for marker effects variance is calculated from the genetic variance and π.
The mean of the prior for the marker effects variance is: 0.168283


A Linear Mixed Model was build using model equations:

y = intercept

Model Information:

Term            C/F          F/R            nLevels
intercept       factor       fixed                1

MCMC Information:

methods                                      BayesC
chain_length                                  60000
burnin                                            0
estimatePi                                     true
estimateScale                                 false
starting_value                                false
printout_frequency                             5000
output_samples_frequency                         20
constraint                                    false
missing_phenotypes                             true
update_priors_frequency                           0

Hyper-parameters Information:

residual variances:        

running MCMC for BayesC...  8%|██▏                      |  ETA: 0:28:25


Posterior means at iteration: 5000
Residual variance: 13.22054
Marker effects variance: 0.685172
π: 0.997


running MCMC for BayesC... 17%|████▏                    |  ETA: 0:25:40


Posterior means at iteration: 10000
Residual variance: 13.251121
Marker effects variance: 0.690046
π: 0.997


...

running MCMC for BayesC... 92%|██████████████████████▉  |  ETA: 0:02:16


Posterior means at iteration: 55000
Residual variance: 13.23267
Marker effects variance: 0.695809
π: 0.997


running MCMC for BayesC...100%|█████████████████████████|  ETA: 0:00:00


Posterior means at iteration: 60000
Residual variance: 13.232632
Marker effects variance: 0.694925
π: 0.997


running MCMC for BayesC...100%|█████████████████████████| Time: 0:27:06


In [12]:
keys(out)

Base.KeySet for a Dict{Any,Any} with 6 entries. Keys:
  "Posterior mean of marker effects"
  "EBV_y"
  "Posterior mean of residual variance"
  "Posterior mean of marker effects variance"
  "Posterior mean of location parameters"
  "Posterior mean of Pi"

In [13]:
out["EBV_y"]

,ID,Estimate,PEV
,Any,Any,Float64
1,1,0.483094,2.19283
2,2,2.4028,2.08013
3,3,-0.511776,2.55766
4,4,8.06149,2.39819
5,5,0.246964,3.00756
6,6,-0.143648,2.78795
7,7,3.59812,2.12147
8,8,0.613668,2.42555
9,9,0.144479,2.14744


In [14]:
res = GWAS("MCMCSamples_marker_effects_y.txt";header=true)

21834×2 Array{Any,2}:
 "1"      0.00333333 
 "2"      0.001      
 "3"      0.001      
 "4"      0.000666667
 "5"      0.000666667
 "6"      0.000666667
 "7"      0.00133333 
 "8"      0.00133333 
 "9"      0.0        
 "10"     0.000333333
 "11"     0.000333333
 "12"     0.001      
 "13"     0.000333333
 ⋮                   
 "21823"  0.00266667 
 "21824"  0.00166667 
 "21825"  0.0        
 "21826"  0.00133333 
 "21827"  0.000666667
 "21828"  0.001      
 "21829"  0.000666667
 "21830"  0.000666667
 "21831"  0.002      
 "21832"  0.000333333
 "21833"  0.0        
 "21834"  0.00166667 

In [15]:
[res[posQTL,:] beta  out["Posterior mean of marker effects"][posQTL,2]]

40×4 Array{Any,2}:
 "8729"   0.341333      1.13899     0.381651   
 "18201"  0.372333      0.742589    0.350259   
 "16771"  0.000333333  -0.416029   -1.39222e-5 
 "14237"  0.006        -0.68567    -0.00404859 
 "11291"  0.0183333    -1.25694    -0.0234683  
 "15837"  0.001         0.060424    1.78569e-5 
 "17008"  0.00266667   -0.333503   -0.000549884
 "16115"  0.002         0.457435    0.000240114
 "1681"   0.000333333   0.0351858   1.2621e-5  
 "15679"  0.275         2.7329      0.613486   
 "2790"   0.00633333   -0.692791   -0.00436163 
 "20843"  0.00166667   -0.411793   -0.000869606
 "19915"  0.00133333   -0.146152   -0.000408079
 ⋮                                             
 "11391"  0.000666667  -0.132497   -0.000131677
 "17889"  0.0343333     0.426983    0.0210134  
 "7815"   0.258667     -1.64235    -0.360953   
 "18247"  0.215667     -1.10174    -0.245908   
 "21776"  0.00366667   -1.16958    -0.00204715 
 "1206"   0.00266667    0.15797     0.000668019
 "6447"   0.198      

In [16]:
winVar = GWAS("MCMCSamples_marker_effects_y.txt",model.output_genotypes;header=true,window_size=100,threshold=0.001)

Compute the posterior probability of association of the genomic window that explains more than 0.001 of the total genetic variance


,wStart,wEnd,wSize,prGenVar,WPPA,PPA_t
,Int64,Int64,Int64,Float64,Float64,Float64
1,7801,7900,100,5.61,1.0,1.0
2,15601,15700,100,19.48,1.0,1.0
3,20201,20300,100,6.6,1.0,1.0
4,20501,20600,100,7.57,0.999667,0.999917
5,8701,8800,100,4.71,0.995,0.998933
6,6401,6500,100,3.89,0.985,0.996611
7,3801,3900,100,4.08,0.979,0.994095
8,18201,18300,100,5.89,0.962333,0.990125
9,3101,3200,100,7.78,0.956333,0.98637


In [17]:
sum(winVar[:prGenVar])

104.65999999999997

In [18]:
sortPosQTL = sort(posQTL);

In [22]:
PPA = 0.34
bigPPA = winVar[PPA .<= winVar[:WPPA],: ]

lowPos  = [findlast(sortPosQTL .<= row[1]) for row in eachrow(bigPPA)] 
highPos = [findfirst(sortPosQTL .>= row[2]) for row in eachrow(bigPPA)]   
wPos = [findfirst(bigPPA[i,1] .<= sortPosQTL .< bigPPA[i,2]) for i=1:size(bigPPA,1) ]

lowQTL  = [i == nothing ? 0 : sortPosQTL[i] for i in lowPos]
highQTL = [i == nothing ? 0 : sortPosQTL[i] for i in highPos]
wQTL    = [i == nothing ? 0 : sortPosQTL[i] for i in wPos]

res = DataFrame(
    wStart = bigPPA[:wStart],
    wEnd = bigPPA[:wEnd],
    wQTL = wQTL,
    oQTL = min.(bigPPA[:wStart]-lowQTL,highQTL-bigPPA[:wEnd]),
    prVar  = bigPPA[:prGenVar],
    WPPA   = bigPPA[:WPPA],
    PPA_t = bigPPA[:PPA_t]
    )

,wStart,wEnd,wQTL,oQTL,prVar,WPPA,PPA_t
,Int64,Int64,Int64,Int64,Float64,Float64,Float64
1,7801,7900,7815,96,5.61,1.0,1.0
2,15601,15700,15679,137,19.48,1.0,1.0
3,20201,20300,20210,252,6.6,1.0,1.0
4,20501,20600,20589,243,7.57,0.999667,0.999917
5,8701,8800,8729,228,4.71,0.995,0.998933
6,6401,6500,6447,1205,3.89,0.985,0.996611
7,3801,3900,3843,355,4.08,0.979,0.994095
8,18201,18300,18201,0,5.89,0.962333,0.990125
9,3101,3200,3139,311,7.78,0.956333,0.98637


In [23]:
(1 - 8/28)*100

71.42857142857143

In [21]:
(1 - 1/15)*100

93.33333333333333